# Données de R&D de quelques pays de l'OCDE
Pour lancer le script, sélectionner une cellule et appuyer sur le bouton play, ou aller dans Run > Run all cells

## Préliminaire

In [ ]:
import subprocess
import sys

# Fonction pour installer un module
def install_and_import(package):
    try:
        __import__(package)
    except ImportError:
        subprocess.check_call([sys.executable, "-m", "pip", "install", package])
        __import__(package)

# Installer les modules nécessaires
install_and_import('pandas')
install_and_import('ipywidgets')
install_and_import('plotnine')

## Dépense par secteur

In [ ]:
import subprocess
import sys

# Fonction pour installer un module
def install_and_import(package):
    try:
        __import__(package)
    except ImportError:
        subprocess.check_call([sys.executable, "-m", "pip", "install", package])
        __import__(package)

# Installer les modules nécessaires
install_and_import('pandas')
install_and_import('plotnine')
install_and_import('ipywidgets')

import pandas as pd
import plotnine as p9
import ipywidgets as widgets
from IPython.display import display, clear_output

# Charger les données
OCDE = pd.read_csv("OCDE.csv")

# Widget pour sélectionner les pays
ref_area_widget = widgets.SelectMultiple(
    options=OCDE["REF_AREA"].unique(),
    description="Régions",
)

# Widgets pour entrer les dates min et max
time_period_min_widget = widgets.BoundedIntText(
    value=2000, min=2000, max=2023, description="Date min"
)

time_period_max_widget = widgets.BoundedIntText(
    value=2023, min=2000, max=2023, description="Date max"
)

# Widget pour sélectionner la métrique
metric_widget = widgets.Dropdown(
    options=[("Part du PIB", "share"), ("Part de la R&D", "share_sect")],
    value="share_sect",
    description="Métrique",
)

# Fonction pour tracer le graphique
output = widgets.Output()

def plot_data(ref_area, time_period_min, time_period_max, metric):
    with output:
        clear_output(wait=True)

        # Filtrer les données selon la sélection de l'utilisateur
        filtered_data = OCDE[
            (OCDE["REF_AREA"].isin(ref_area))
            & (OCDE["TIME_PERIOD"] >= time_period_min)
            & (OCDE["TIME_PERIOD"] <= time_period_max)
        ]

        # Vérifier si le dataframe filtré est vide
        if filtered_data.empty:
            print("Aucune donnée disponible pour ces critères.")
            return

        # Définir les couleurs pour SECT_PERF
        colors = {
            "_T": "gray",
            "GOV": "#882255",
            "BES": "#009988",
            "HES": "#CCBB44",
        }

        # Créer un graphique avec `plotnine`
        p = (
            p9.ggplot(filtered_data, p9.aes(x="TIME_PERIOD", y=metric, fill="SECT_PERF"))
            + p9.geom_bar(stat="identity", position="dodge")  # Pour séparer les barres
            + p9.facet_wrap("~REF_AREA", scales="fixed")  # Un graphique par pays
            + p9.labs(x="Année", y=metric, fill="Secteur")
            + p9.theme_minimal()
            + p9.theme(figure_size=(25, 8))  # Ajuster la largeur du graphique à 25
            + p9.scale_fill_manual(values=colors)
        )

        # Afficher le graphique
        display(p)

# Interface interactive
interactive_plot = widgets.interactive(
    plot_data,
    ref_area=ref_area_widget,
    time_period_min=time_period_min_widget,
    time_period_max=time_period_max_widget,
    metric=metric_widget,
)

# Affichage des widgets et du bouton dans un conteneur séparé
input_panel = widgets.VBox([
    ref_area_widget, 
    time_period_min_widget, 
    time_period_max_widget, 
    metric_widget
])

# Affichage du panneau de saisie (widgets)
display(input_panel)

# Affichage du panneau de sortie (graphique) dans un conteneur séparé
display(output)

# Lier la fonction de mise à jour au widget interactif
interactive_plot.observe(lambda change: plot_data(ref_area_widget.value, time_period_min_widget.value, time_period_max_widget.value, metric_widget.value), names='value')

## Dépense par secteur et type de financements

In [ ]:
import pandas as pd
import ipywidgets as widgets
from IPython.display import display, clear_output
from plotnine import ggplot, aes, geom_col, labs, theme_minimal, facet_grid, scale_fill_manual, theme

# 📌 Charger les données OCDE
OCDE_financements = pd.read_csv("OCDE_financements.csv")

# 📌 Mapping des labels pour SECT_PERF (exécution de la R&D)
sect_perf_labels = {
    "BES": "Entreprises",
    "HES": "Ens. sup.",
    "GOV": "État",
    "PNP": "Privé non luc.",
    "_T": "Total"
}

# 📌 Mapping des labels pour SECT_FUND (source de financement)
sect_fund_labels = {
    "BES": "Entreprises",
    "HES": "Enseignement supérieur",
    "GOV": "État",
    "PNP": "Privé non lucratif",
    "ROW": "Reste du monde",
    "ROW_BES": "dont entreprises",  # ✅ Sous-catégorie de ROW
    "_T": "Total"
}

# 📌 Liste des pays
pays_options = OCDE_financements["REF_AREA"].unique()

# 📌 Cases à cocher pour les pays
checkboxes = [widgets.Checkbox(value=False, description=p) for p in pays_options]
checkbox_widget = widgets.VBox(checkboxes)

# 📌 Inputs pour la période
min_year = widgets.IntText(value=2000, description="Année min :")
max_year = widgets.IntText(value=2023, description="Année max :")

# 📌 Menu déroulant pour la métrique
metric_widget = widgets.Dropdown(
    options={"Part du PIB": "share_financements_PIB", "Part de la R&D": "share_financements"},
    value="share_financements",
    description="Métrique"
)

# 📌 Menus déroulants pour SECT_PERF et SECT_FUND
sect_perf_options = ["Entreprises", "Ens. sup.", "État", "Privé non luc.", "Total"]
sect_fund_options = ["Entreprises", "Enseignement supérieur", "État", "Privé non lucratif", "Reste du monde", "dont entreprises"]

sect_perf_widget = widgets.SelectMultiple(
    options=sect_perf_options,
    value=sect_perf_options,
    description="Secteur de performance",
    disabled=False
)

sect_fund_widget = widgets.SelectMultiple(
    options=sect_fund_options,
    value=sect_fund_options,
    description="Secteur de financement",
    disabled=False
)

# 📌 Bouton de mise à jour
update_button = widgets.Button(description="Mettre à jour")

# 📌 Fonction pour récupérer les pays sélectionnés
def get_selected():
    return [cb.description for cb in checkboxes if cb.value]

# 📌 Fonction de mise à jour du graphique
output = widgets.Output()

def update_plot(b):
    with output:
        clear_output(wait=True)

        # Sélections utilisateur
        selected_pays = get_selected()
        min_date = min_year.value
        max_date = max_year.value
        metric = metric_widget.value
        selected_sect_perf = sect_perf_widget.value
        selected_sect_fund = sect_fund_widget.value

        # Filtrage des données
        df_filtered = OCDE_financements[
            (OCDE_financements["REF_AREA"].isin(selected_pays)) &
            (OCDE_financements["TIME_PERIOD"] >= min_date) &
            (OCDE_financements["TIME_PERIOD"] <= max_date)
        ].copy()

        # Vérifier qu'on a des données
        if df_filtered.empty:
            print("Aucune donnée disponible pour cette sélection.")
            return

        # 📌 Appliquer les labels
        df_filtered["SECT_PERF"] = df_filtered["SECT_PERF"].map(sect_perf_labels)
        df_filtered["SECT_FUND"] = df_filtered["SECT_FUND"].map(sect_fund_labels)

        # 📌 Filtrage pour SECT_PERF et SECT_FUND selon les choix utilisateurs
        df_filtered = df_filtered[df_filtered["SECT_PERF"].isin(selected_sect_perf)]
        df_filtered = df_filtered[df_filtered["SECT_FUND"].isin(selected_sect_fund)]

        # 📌 Catégories ordonnées pour la légende
        df_filtered["SECT_FUND"] = pd.Categorical(
            df_filtered["SECT_FUND"],
            categories=["Entreprises", "État", "Enseignement supérieur", "Privé non lucratif", 
                        "Reste du monde", "dont entreprises"],
            ordered=True
        )

        # 📌 Création du graphique avec plotnine
        plot = (
            ggplot(df_filtered, aes(x="TIME_PERIOD", y=metric, fill="SECT_FUND")) +
            geom_col(position="dodge") +
            labs(y=metric, x="Année", fill="Source de financement") +
            theme_minimal() +
            facet_grid("SECT_PERF ~ REF_AREA") +  # ✅ Un graphique par pays et catégorie SECT_PERF
            scale_fill_manual(
                values={
                    "Entreprises": "#009988", 
                    "État": "#882255", 
                    "Enseignement supérieur": "#CCBB44",
                    "Privé non lucratif": "#7BA05B",
                    "Reste du monde": "#004488",
                    "dont entreprises": "#7FA1C3"
                }
            ) +
            theme(
                figure_size=(25, 8)  # Ajustez les dimensions ici
            )
        )

        # 📌 Affichage du graphique (plotnine gère l'affichage automatiquement dans Jupyter)
        display(plot)

# 📌 Lier le bouton à la fonction de mise à jour
update_button.on_click(update_plot)

# 📌 Affichage des widgets et du bouton dans un conteneur séparé
input_panel = widgets.VBox([
    checkbox_widget, 
    min_year, 
    max_year, 
    metric_widget, 
    sect_perf_widget, 
    sect_fund_widget, 
    update_button
])

# Affichage du panneau de saisie (widgets)
display(input_panel)

# 📌 Affichage du panneau de sortie (graphique) dans un conteneur séparé
display(output)